<a href="https://colab.research.google.com/github/banea/DeceptionDetection/blob/master/AMT_Result_Processing_analysis_generic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import ast
import re
import json

In [ ]:
data_path = '/home/Resources/datasets/NeedIdentification/AMT/raw/'
df = pd.read_csv(data_path + 'Batch_4223503_batch_results.csv')
display(df.head())
df.info()
# drop useless columns (should work for different HITs as well, as we are dropping from the beginning and end)
cols_to_drop = [*range(0,15)] + [*range(16,23)] + [*range(24,27)] + [*range(len(df.columns)-2, len(df.columns))]
print ("Dropping columns with indeces: %s" % ",".join(map(str,cols_to_drop)))
df.drop(axis=1, columns=df.iloc[:, cols_to_drop], inplace=True)
df.info()

,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,...,Input.text_9,Input.id_10,Input.user_10,Input.text_10,Input.id_11,Input.user_11,Input.text_11,Answer.taskAnswers,Approve,Reject
0,3XBXDSS89CKAHOP1JJNICOB68EZXLM,3TD7B5RKTL8U27EB5ZOGAAJ3VSHQZL,Annotate shortages in tweets,An analysis of shortage-focused tweets as they...,"tweet analysis, text, text extraction, shortag...",$0.60,Mon Oct 19 22:48:59 PDT 2020,3,BatchId:4223503;OriginalHitTemplateId:928390909;,3600,...,Have you recovered from #COVID19? @my1blood is...,1.291460e+18,SAIndiana,Are you having trouble paying your utility bil...,1.291450e+18,ProtectCare,A recent report found that hospitals will lose...,"[{""tweet1.1-shortage"":{""false"":false,""true"":tr...",NaN,NaN
1,3XBXDSS89CKAHOP1JJNICOB68EZXLM,3TD7B5RKTL8U27EB5ZOGAAJ3VSHQZL,Annotate shortages in tweets,An analysis of shortage-focused tweets as they...,"tweet analysis, text, text extraction, shortag...",$0.60,Mon Oct 19 22:48:59 PDT 2020,3,BatchId:4223503;OriginalHitTemplateId:928390909;,3600,...,Have you recovered from #COVID19? @my1blood is...,1.291460e+18,SAIndiana,Are you having trouble paying your utility bil...,1.291450e+18,ProtectCare,A recent report found that hospitals will lose...,"[{""tweet1.1-shortage"":{""false"":false,""true"":tr...",NaN,NaN
2,3XBXDSS89CKAHOP1JJNICOB68EZXLM,3TD7B5RKTL8U27EB5ZOGAAJ3VSHQZL,Annotate shortages in tweets,An analysis of shortage-focused tweets as they...,"tweet analysis, text, text extraction, shortag...",$0.60,Mon Oct 19 22:48:59 PDT 2020,3,BatchId:4223503;OriginalHitTemplateId:928390909;,3600,...,Have you recovered from #COVID19? @my1blood is...,1.291460e+18,SAIndiana,Are you having trouble paying your utility bil...,1.291450e+18,ProtectCare,A recent report found that hospitals will lose...,"[{""tweet1.1-shortage"":{""false"":false,""true"":tr...",NaN,NaN
3,3GL25Y6857VUN67MJVOXC1QV8JPXMO,3TD7B5RKTL8U27EB5ZOGAAJ3VSHQZL,Annotate shortages in tweets,An analysis of shortage-focused tweets as they...,"tweet analysis, text, text extraction, shortag...",$0.60,Mon Oct 19 22:48:59 PDT 2020,3,BatchId:4223503;OriginalHitTemplateId:928390909;,3600,...,Had a positive #COVID19 test? Recovered? If yo...,1.291330e+18,PHE South West,People in the #SouthWest who are obese are twi...,1.291140e+18,ACS CAN Hawaii Pacific,Cancer patients and survivors need your help. ...,"[{""13-comments"":""none"",""tweet1.1-shortage"":{""f...",NaN,NaN
4,3GL25Y6857VUN67MJVOXC1QV8JPXMO,3TD7B5RKTL8U27EB5ZOGAAJ3VSHQZL,Annotate shortages in tweets,An analysis of shortage-focused tweets as they...,"tweet analysis, text, text extraction, shortag...",$0.60,Mon Oct 19 22:48:59 PDT 2020,3,BatchId:4223503;OriginalHitTemplateId:928390909;,3600,...,Had a positive #COVID19 test? Recovered? If yo...,1.291330e+18,PHE South West,People in the #SouthWest who are obese are twi...,1.291140e+18,ACS CAN Hawaii Pacific,Cancer patients and survivors need your help. ...,"[{""tweet1.1-shortage"":{""false"":false,""true"":tr...",NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 66 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   HITId                        36 non-null     object 
 1   HITTypeId                    36 non-null     object 
 2   Title                        36 non-null     object 
 3   Description                  36 non-null     object 
 4   Keywords                     36 non-null     object 
 5   Reward                       36 non-null     object 
 6   CreationTime                 36 non-null     object 
 7   MaxAssignments               36 non-null     int64  
 8   RequesterAnnotation          36 non-null     object 
 9   AssignmentDurationInSeconds  36 non-null     int64  
 10  AutoApprovalDelayInSeconds   36 non-null     int64  
 11  Expiration                   36 non-null     object 
 12  NumberOfSimilarHITs          0 non-null      float64
 13  LifetimeInSeconds     

## Unpacking individual answers from Answer.taskAnswers

In [ ]:
raw_entry = df.loc[0, 'Answer.taskAnswers']
display (json.loads(raw_entry[1:-1])) # raw entry contains a list, and we want
raw_annot = json.loads(raw_entry[1:-1])
# it to be parsed as a dict

primary_annot_columns = []
for category in raw_annot:
  if re.match (r'tweet\d+\.1-shortage', category): 
    print (category)
    primary_annot_columns.append(category)

for column in primary_annot_columns:
  df["raw_"+ column] = df['Answer.taskAnswers'].apply(lambda x: json.loads(x[1:-1])[column])
#  # the label of shortage is true when 'true' == True, otherwise it is False
  df[column] = df["raw_"+ column].apply(lambda x: x['true']) 

display(df)

{'tweet1.1-shortage': {'false': False, 'true': True},
 'tweet1.2-type': {'education': False,
  'food': True,
  'medical': False,
  'money': False,
  'non-medical-equipment': False,
  'other': False,
  'shelter': False,
  'volunteers': False},
 'tweet1.3-entity-generic-group': {'generic-group': False},
 'tweet1.3-entity-individual': {'individual': True},
 'tweet1.3-entity-legal-entity': {'legal-entity': False},
 'tweet10.1-shortage': {'false': False, 'true': True},
 'tweet10.2-type': {'education': False,
  'food': False,
  'medical': False,
  'money': False,
  'non-medical-equipment': False,
  'other': True,
  'shelter': False,
  'volunteers': True},
 'tweet10.2-type-other': 'plasma',
 'tweet10.3-entity-generic-group': {'generic-group': False},
 'tweet10.3-entity-individual': {'individual': True},
 'tweet10.3-entity-legal-entity': {'legal-entity': False},
 'tweet11.1-shortage': {'false': False, 'true': True},
 'tweet11.2-type': {'education': False,
  'food': False,
  'medical': False,
 

tweet1.1-shortage
tweet10.1-shortage
tweet11.1-shortage
tweet12.1-shortage
tweet2.1-shortage
tweet3.1-shortage
tweet4.1-shortage
tweet5.1-shortage
tweet6.1-shortage
tweet7.1-shortage
tweet8.1-shortage
tweet9.1-shortage


,WorkerId,WorkTimeInSeconds,Input.id_0,Input.user_0,Input.text_0,Input.id_1,Input.user_1,Input.text_1,Input.id_2,Input.user_2,...,raw_tweet5.1-shortage,tweet5.1-shortage,raw_tweet6.1-shortage,tweet6.1-shortage,raw_tweet7.1-shortage,tweet7.1-shortage,raw_tweet8.1-shortage,tweet8.1-shortage,raw_tweet9.1-shortage,tweet9.1-shortage
0,A1M24593XO5Z3C,718,1.291740e+18,SurreyCares,"Another $5,000 went to Surrey North Delta Meal...",1.291730e+18,Wyatt First Aid,The HSE advise that FAW certificates which exp...,1.291700e+18,Upanshu,...,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True
1,A1T0S48O0CMXBC,2528,1.291740e+18,SurreyCares,"Another $5,000 went to Surrey North Delta Meal...",1.291730e+18,Wyatt First Aid,The HSE advise that FAW certificates which exp...,1.291700e+18,Upanshu,...,"{'false': True, 'true': False}",False,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True
2,A1ET2J1PIP0RGO,944,1.291740e+18,SurreyCares,"Another $5,000 went to Surrey North Delta Meal...",1.291730e+18,Wyatt First Aid,The HSE advise that FAW certificates which exp...,1.291700e+18,Upanshu,...,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True
3,A2Q4X0C4SZ1QEY,1713,1.291450e+18,What About Us The Musical,#Artists worldwide have been affected dispropo...,1.291440e+18,Harvest Hope Food Bank,Thanks to @CVSinAction for your generous contr...,1.291440e+18,World Vision UK,...,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': True, 'true': False}",False,"{'false': False, 'true': True}",True
4,A1ET2J1PIP0RGO,1536,1.291450e+18,What About Us The Musical,#Artists worldwide have been affected dispropo...,1.291440e+18,Harvest Hope Food Bank,Thanks to @CVSinAction for your generous contr...,1.291440e+18,World Vision UK,...,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True
5,A31X0GSY38JNZY,3129,1.291450e+18,What About Us The Musical,#Artists worldwide have been affected dispropo...,1.291440e+18,Harvest Hope Food Bank,Thanks to @CVSinAction for your generous contr...,1.291440e+18,World Vision UK,...,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': True, 'true': False}",False,"{'false': True, 'true': False}",False,"{'false': True, 'true': False}",False
6,A1ET2J1PIP0RGO,396,1.291110e+18,Children's Rights,"Thank you, @RepDonBacon for recognizing the ne...",1.291100e+18,Memphis Chamber,"Due to #COVID19, there has been a rapid decrea...",1.291050e+18,Forest Ray,...,"{'false': True, 'true': False}",False,"{'false': True, 'true': False}",False,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': True, 'true': False}",False
7,A1T0S48O0CMXBC,3073,1.291110e+18,Children's Rights,"Thank you, @RepDonBacon for recognizing the ne...",1.291100e+18,Memphis Chamber,"Due to #COVID19, there has been a rapid decrea...",1.291050e+18,Forest Ray,...,"{'false': True, 'true': False}",False,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True
8,AIRHEW6W1BIZG,677,1.291110e+18,Children's Rights,"Thank you, @RepDonBacon for recognizing the ne...",1.291100e+18,Memphis Chamber,"Due to #COVID19, there has been a rapid decrea...",1.291050e+18,Forest Ray,...,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True
9,A2RQKVTQKESIW

In [ ]:
print ("Annotation overview: ")
hits_to_drop = []
# check number of turkers who gave the same label throughout
for idx, hit in df.iterrows():
  cnt_True = 0
  cnt_False = 0
  for column in primary_annot_columns:
    if hit[column] == True:
      cnt_True+=1
    else:
      cnt_False+=1
  matching_annot = max(cnt_True, cnt_False)
  if matching_annot > len(primary_annot_columns) * 0.7:
    hits_to_drop.append(idx)
  print ("%d\t T: %d\t F: %d\t %d" % (idx, cnt_True, cnt_False, hit["WorkTimeInSeconds"]))

print ("Dropping %d outlier hits: %s" % (len(hits_to_drop), ",".join(map(str, hits_to_drop))))
df.drop(hits_to_drop, inplace=True)
df.info()
display(df)

Annotation overview: 
0	 T: 10	 F: 2	 718
1	 T: 9	 F: 3	 2528
2	 T: 12	 F: 0	 944
3	 T: 9	 F: 3	 1713
4	 T: 12	 F: 0	 1536
5	 T: 8	 F: 4	 3129
6	 T: 5	 F: 7	 396
7	 T: 10	 F: 2	 3073
8	 T: 11	 F: 1	 677
9	 T: 11	 F: 1	 756
10	 T: 11	 F: 1	 539
11	 T: 10	 F: 2	 2968
12	 T: 8	 F: 4	 1116
13	 T: 11	 F: 1	 3245
14	 T: 12	 F: 0	 1164
15	 T: 12	 F: 0	 1577
16	 T: 10	 F: 2	 2855
17	 T: 12	 F: 0	 249
18	 T: 12	 F: 0	 1851
19	 T: 9	 F: 3	 3393
20	 T: 10	 F: 2	 1092
21	 T: 8	 F: 4	 582
22	 T: 8	 F: 4	 3466
23	 T: 8	 F: 4	 3139
24	 T: 12	 F: 0	 1120
25	 T: 12	 F: 0	 1982
26	 T: 12	 F: 0	 3457
27	 T: 12	 F: 0	 561
28	 T: 12	 F: 0	 1217
29	 T: 11	 F: 1	 555
30	 T: 11	 F: 1	 866
31	 T: 12	 F: 0	 3547
32	 T: 7	 F: 5	 1869
33	 T: 12	 F: 0	 2065
34	 T: 12	 F: 0	 1053
35	 T: 12	 F: 0	 1197
Dropping 29 outlier hits: 0,1,2,3,4,7,8,9,10,11,13,14,15,16,17,18,19,20,24,25,26,27,28,29,30,31,33,34,35
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 5 to 32
Data columns (total 63 columns):
 #   Colum

,WorkerId,WorkTimeInSeconds,Input.id_0,Input.user_0,Input.text_0,Input.id_1,Input.user_1,Input.text_1,Input.id_2,Input.user_2,...,raw_tweet5.1-shortage,tweet5.1-shortage,raw_tweet6.1-shortage,tweet6.1-shortage,raw_tweet7.1-shortage,tweet7.1-shortage,raw_tweet8.1-shortage,tweet8.1-shortage,raw_tweet9.1-shortage,tweet9.1-shortage
5,A31X0GSY38JNZY,3129,1.291450e+18,What About Us The Musical,#Artists worldwide have been affected dispropo...,1.291440e+18,Harvest Hope Food Bank,Thanks to @CVSinAction for your generous contr...,1.291440e+18,World Vision UK,...,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': True, 'true': False}",False,"{'false': True, 'true': False}",False,"{'false': True, 'true': False}",False
6,A1ET2J1PIP0RGO,396,1.291110e+18,Children's Rights,"Thank you, @RepDonBacon for recognizing the ne...",1.291100e+18,Memphis Chamber,"Due to #COVID19, there has been a rapid decrea...",1.291050e+18,Forest Ray,...,"{'false': True, 'true': False}",False,"{'false': True, 'true': False}",False,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': True, 'true': False}",False
12,A1T0S48O0CMXBC,1116,1.290320e+18,"El Paso County, Colorado",Are you or someone you know struggling due to ...,1.290290e+18,Sarah Boyack,@AnthonyNolan is appealing for urgent donation...,1.290250e+18,Financial Education Basics,...,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True
21,A59ZTO26IMTZ4,582,1.288550e+18,HemophiliaFederation,"As #COVID19 continues, patients could be force...",1.288550e+18,Carrie TX,If your district is below or not: 1. Verify yo...,1.288540e+18,Oralia Loza,...,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': True, 'true': False}",False,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True
22,A1ET2J1PIP0RGO,3466,1.288550e+18,HemophiliaFederation,"As #COVID19 continues, patients could be force...",1.288550e+18,Carrie TX,If your district is below or not: 1. Verify yo...,1.288540e+18,Oralia Loza,...,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': True, 'true': False}",False
23,A292N98YNVO16T,3139,1.288550e+18,HemophiliaFederation,"As #COVID19 continues, patients could be force...",1.288550e+18,Carrie TX,If your district is below or not: 1. Verify yo...,1.288540e+18,Oralia Loza,...,"{'false': True, 'true': False}",False,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True
32,AIHUIAQ4922K3,1869,1.290650e+18,International Institute of New England,"We are all in, all together, for our #refugee ...",1.290640e+18,Muhammad Abubakar Ali,Kenya received billions to address the #COVID1...,1.290610e+18,Gary Kirsten Foundation,...,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True,"{'false': True, 'true': False}",False,"{'false': False, 'true': True}",True,"{'false': False, 'true': True}",True


## Create gs dataframe with complete and truncated tweet id

In [ ]:
df_batch = pd.read_csv(data_path + 'out1.csv')
df_batch.tail()

,id_0,user_0,text_0
149,1288630189606318080,Red Cross Los Angeles,"We're so glad you ""jumped"" at the opportunity ..."
150,1288581198839599105,Idaho Red Cross,"Right now, we're distributing convalescent pla..."
151,1288556427674517504,Community Foundation of South Jersey,"To date, we have raised $1,075,000 for the SJ ..."
152,1288547375846481921,Nation's Best CPR - #wearamask,EMERGENCY SHORTAGE! #COVID19 Convalescent Plas...
153,1288514371514064896,Hillingdon Hospitals #BeSmartStayApart,"If you've had #COVID19, you may be eligible to..."


In [ ]:
df_gold = pd.read_csv(data_path + 'small_batch_gs.csv')
df_gold = df_gold.drop([0]).reset_index()
df_gold.tail

<bound method NDFrame.tail of      index          id_0                                  user_0  \
0        1  1.291740e+18                             SurreyCares   
1        2  1.291730e+18                         Wyatt First Aid   
2        3  1.291700e+18                                 Upanshu   
3        4  1.291700e+18                       Trafford SENDIASS   
4        5  1.291650e+18                        Rochdale Council   
..     ...           ...                                     ...   
149    150  1.288630e+18                   Red Cross Los Angeles   
150    151  1.288580e+18                         Idaho Red Cross   
151    152  1.288560e+18    Community Foundation of South Jersey   
152    153  1.288550e+18        Nation's Best CPR - #wearamask     
153    154  1.288510e+18  Hillingdon Hospitals #BeSmartStayApart   

                                                text_0   q1  \
0    Another $5,000 went to Surrey North Delta Meal...  1.0   
1    The HSE advise that FA

In [ ]:
df_batch['TweetId'] = df_gold['id_0']
df_batch['q1'] = df_gold['q1']
df_batch['q2'] = df_gold['q2']
df_batch['q3'] = df_gold['q3']

display (df_batch.head())
df_batch.info()

,id_0,user_0,text_0,TweetId,q1,q2,q3
0,1291736604092989441,SurreyCares,"Another $5,000 went to Surrey North Delta Meal...",1.291740e+18,1.0,other: nonspecific,legal-entity
1,1291727439979307008,Wyatt First Aid,The HSE advise that FAW certificates which exp...,1.291730e+18,0.0,NaN,NaN
2,1291704008101384192,Upanshu,Name- Urmila Devi Age- 72 Yrs Blood type- B+ H...,1.291700e+18,1.0,medical,individual
3,1291698370562723846,Trafford SENDIASS,Support is in place for residents who need it ...,1.291700e+18,0.0,NaN,NaN
4,1291653122213810178,Rochdale Council,Anyone who is struggling and needs help to sta...,1.291650e+18,0.0,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id_0     154 non-null    int64  
 1   user_0   154 non-null    object 
 2   text_0   154 non-null    object 
 3   TweetId  154 non-null    float64
 4   q1       154 non-null    float64
 5   q2       72 non-null     object 
 6   q3       72 non-null     object 
dtypes: float64(2), int64(1), object(4)
memory usage: 8.5+ KB


## Now we need to extract individual tweet answers from our AMT batch df

In [ ]:
def new_AMT_row(row: pd.DataFrame, tweet_id: pd.DataFrame, primary_annot_columns: list) -> pd.DataFrame:
    '''
    Takes in a row of the AMT batched data and breaks it into multiple rows 
    '''
    return_list = []
    pattern = re.compile("tweet(\d+)\.\d+-shortage")
    #display (row)
    for column in primary_annot_columns:
      # column format: "tweet8.1-shortage" !this count starts at 1 not at 0!!!
      id = str( int(re.search(pattern, column).group(1)) - 1)
      #print (id)
      #print (type(id))
      temp_dict = dict()
      temp_dict['shortageLabel'] = row[column]
      temp_dict['WorkerId'] = row.WorkerId
      temp_dict['TweetId_e'] = row['Input.id_'+id]
      temp_dict['TweetUser'] = row['Input.user_'+id]
      temp_dict['TweetText'] = row['Input.text_'+id]
      return_list.append(temp_dict)
    
    return pd.DataFrame(return_list)


df_new = []
for i, v in df.iterrows():
    df_new.append(new_AMT_row(v, df_batch, primary_annot_columns))
df_new = pd.concat(df_new)
display (df_new.head())
df_new.info()

,shortageLabel,WorkerId,TweetId_e,TweetUser,TweetText
0,True,A31X0GSY38JNZY,1.291450e+18,What About Us The Musical,#Artists worldwide have been affected dispropo...
1,True,A31X0GSY38JNZY,1.291340e+18,U.S. FDA,Had a positive #COVID19 test? Recovered? If yo...
2,False,A31X0GSY38JNZY,1.291330e+18,PHE South West,People in the #SouthWest who are obese are twi...
3,True,A31X0GSY38JNZY,1.291140e+18,ACS CAN Hawaii Pacific,Cancer patients and survivors need your help. ...
4,True,A31X0GSY38JNZY,1.291440e+18,Harvest Hope Food Bank,Thanks to @CVSinAction for your generous contr...


<class 'pandas.core.frame.DataFrame'>
Int64Index: 84 entries, 0 to 11
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   shortageLabel  84 non-null     bool   
 1   WorkerId       84 non-null     object 
 2   TweetId_e      84 non-null     float64
 3   TweetUser      84 non-null     object 
 4   TweetText      84 non-null     object 
dtypes: bool(1), float64(1), object(3)
memory usage: 3.4+ KB


In [ ]:
# Join extracted AMT annotations df (one tweet per row) with the annotation df. Since we cannot join on the Tweet_Id bc it is malformed, we join on tweet text.
df_complete = df_new.set_index('TweetText').join(other=df_batch.set_index('text_0'), how='left')
df_complete = df_complete.reset_index().rename(columns={"index": "TweetText", "id_0":"TweetId", 'q1':'gs_shortageLabel'})
df_complete['gs_shortageLabel'] = df_complete['gs_shortageLabel'].replace({1.0: True, 0.0: False})
display(df_complete)

,TweetText,shortageLabel,WorkerId,TweetId_e,TweetUser,TweetId,user_0,TweetId,gs_shortageLabel,q2,q3
0,#Artists worldwide have been affected dispropo...,True,A31X0GSY38JNZY,1.291450e+18,What About Us The Musical,1291446175015542786,What About Us The Musical,1.291450e+18,False,NaN,NaN
1,#COVID19 has increased demand for blood donati...,False,AIHUIAQ4922K3,1.290560e+18,Roche,1290558112173182978,Roche,1.290560e+18,False,NaN,NaN
2,#Jerusalem hospitals network need approx $7mil...,True,A1T0S48O0CMXBC,1.289810e+18,Medical Aid for Palestinians,1289809478573989888,Medical Aid for Palestinians,1.289810e+18,True,medical,generic-group
3,"#sundayvibes Greetings from Weed, California! ...",True,A1T0S48O0CMXBC,1.289890e+18,LEO’sCoastToCoast,1289885776449167360,LEO’sCoastToCoast,1.289890e+18,False,NaN,NaN
4,.@CHI_StLukes Emergency Center in Conroe is ho...,True,AIHUIAQ4922K3,1.290240e+18,"Jim Parisi, BSN, MBA",1290242011425992705,"Jim Parisi, BSN, MBA",1.290240e+18,True,medical,legal-entity
...,...,...,...,...,...,...,...,...,...,...,...
79,"With #lockdown easing, we wanted to pause, ref...",True,A1ET2J1PIP0RGO,1.288500e+18,Start Up Stirling,1288497009398370305,Start Up Stirling,1.288500e+18,False,NaN,NaN
80,"With #lockdown easing, we wanted to pause, ref...",True,A292N98YNVO16T,1.288500e+18,Start Up Stirling,1288497009398370305,Start Up Stirling,1.288500e+18,False,NaN,NaN
81,“Pick Up Grab-and-Go Meals at APS High Schools...,False,A59ZTO26IMTZ4,1.288490e+18,Rep. Day Hochman-Vigil,1288486317416804352,Rep. Day Hochman-Vigil,1.288490e+18,False,NaN,NaN
82,“Pick Up Grab-and-Go Meals at APS High Schools...,True,A1ET2J1PIP0RGO,1.288490e+18,Rep. Day Hochman-Vigil,1288486317416804352,Rep. Day Hochman-Vigil,1.288490e+18,False,NaN,NaN


In [ ]:
from sklearn.metrics import confusion_matrix
y_true = df_complete.gs_shortageLabel
y_pred = df_complete.shortageLabel
confusion_matrix(y_true, y_pred).ravel()

array([30, 28,  2, 24])

## STOP HERE
Understanding the annotation errors for need tweets



In [ ]:
df_need = df_analysis[df_analysis['GoldShortageLabel'] == 1.0]
df_no_need = df_analysis[df_analysis['GoldShortageLabel'] == 0.0]
print ("NEED")
display (df_need.head())
print ("NO NEED")
display (df_no_need.head())

In [ ]:
diff_need = df_need.groupby(by=['TweetId']).sum()
diff_no_need = df_no_need.groupby(by=['TweetId']).sum()
display(diff_need[diff_need['WorkerShortageLabel'] == 3.0].shape)
display(diff_need[diff_need['WorkerShortageLabel'] == 2.0].shape)
display(diff_need[diff_need['WorkerShortageLabel'] == 1.0].shape)
display(diff_need[diff_need['WorkerShortageLabel'] == 0.0].shape)

In [ ]:
df_batch.set_index('id_0', inplace=True)
display (df_batch)

In [ ]:
df_need_2 = diff_need[diff_need['WorkerShortageLabel'] == 2.0]
display (df_need_2)
for idx, row in df_need_2.iterrows():
  display (idx)
  display (df_batch.loc[idx,'text_0'])

## Understanding the annotation errors for no need tweets

In [ ]:
# diff_no_need = df_no_need.groupby(by=['TweetId']).sum()
# note: the higher the number, the more wrong the annotation
display(diff_no_need[diff_no_need['WorkerShortageLabel'] == 3.0].shape)
display(diff_no_need[diff_no_need['WorkerShortageLabel'] == 2.0].shape)
display(diff_no_need[diff_no_need['WorkerShortageLabel'] == 1.0].shape)
display(diff_no_need[diff_no_need['WorkerShortageLabel'] == 0.0].shape)

In [ ]:
df_new['ConsistentLabel'] = df_new.WorkerShortageLabel == df_new.GoldShortageLabel
df_new.head()

In [ ]:
df_new.ConsistentLabel.mean()

In [ ]:
df_new.to_csv('Data/AMT/small_batch_gold_and_annotated.csv')

In [ ]:
df_new[['WorkerId', 'ConsistentLabel']].groupby(['WorkerId']).agg({'ConsistentLabel': ['sum', 'count', 'mean']}).reset_index()

In [ ]:
df_new[['TweetId', 'TweetText', 'ConsistentLabel']].groupby(['TweetId', 'TweetText']).agg(['sum', 'count', 'mean']).reset_index()

In [ ]:
df_new[['WorkerId', 'WorkerShortageLabel', 'GoldShortageLabel']].groupby(['WorkerId']).corr(method='pearson')

In [ ]:
df_new[df_new.TweetId == 1288479455833403392].TweetText.values[0]